# STRUCTURAL ANALYSIS

In [2]:
# Libraries required for the code

import pandas as pd
import statistics
import importlib
from scipy import stats
import math
import numpy as np
from pathlib import Path  
import vcf
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
import Bio.SeqIO
import Bio.AlignIO
from Bio.PDB import *



In [31]:
# Reads in pickle file from VarAlign output, containing all structural data
Structure = pd.read_pickle(r'C:\Users\Adam Martin\OneDrive - University of Dundee\Honours Project\231006a-AM-honours-project-aps\varalign-data-PF00026\pickles\PF00026.22_swissprot.sto_prointvar_structure_table.p.gz')

In [35]:
# Viewing structure table
Structure

,ATOM_A,ATOM_B,Alignment_column_A,Alignment_column_B,B_iso_or_equiv_A,B_iso_or_equiv_B,CATH_dbAccessionId_A,CATH_dbAccessionId_B,CATH_regionEnd_A,CATH_regionEnd_B,...,pdbx_formal_charge_B,type_symbol_A,type_symbol_B,interaction_type,protein_topology,pfam_domain_topology,cath_domain_topology,cath_hierarchical_topology,scop_domain_topology,polymer_topology
0,C,N,230.0,232.0,16.675000,12.560000,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
1,O,N,177.0,232.0,34.367500,12.560000,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
2,O,O,230.0,311.0,16.675000,18.305714,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
3,O,N,300.0,311.0,47.157500,18.305714,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
6,NE2,O,229.0,311.0,16.737778,18.305714,2.40.70.10,2.40.70.10,214.0,214.0,...,?,N,N,Protein-Protein,Homoprotein,Homodomain (Self),Homodomain,level_4,Homodomain,Intrapolymer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222791,NaN,NaN,424.0,NaN,41.920000,NaN,2.40.70.10,NaN,383.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer
2222792,NaN,NaN,120.0,NaN,62.100000,NaN,2.40.70.10,NaN,147.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer
2222793,NaN,NaN,144.0,NaN,56.217778,NaN,2.40.70.10,NaN,147.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer
2222794,NaN,NaN,227.0,NaN,43.244286,NaN,2.40.70.10,NaN,147.0,NaN,...,NaN,N,NaN,NaN,NaN,NaN,Domain-NaN,NaN,NaN,Interpolymer


In [36]:
# All the different columns containing lots of information
Structure.columns.to_list()

['ATOM_A',
 'ATOM_B',
 'Alignment_column_A',
 'Alignment_column_B',
 'B_iso_or_equiv_A',
 'B_iso_or_equiv_B',
 'CATH_dbAccessionId_A',
 'CATH_dbAccessionId_B',
 'CATH_regionEnd_A',
 'CATH_regionEnd_B',
 'CATH_regionId_A',
 'CATH_regionId_B',
 'CATH_regionResNum_A',
 'CATH_regionResNum_B',
 'CATH_regionStart_A',
 'CATH_regionStart_B',
 'CHAIN_A',
 'CHAIN_B',
 'Cartn_x_A',
 'Cartn_x_B',
 'Cartn_y_A',
 'Cartn_y_B',
 'Cartn_z_A',
 'Cartn_z_B',
 'DIST',
 'Ensembl_dbAccessionId_A',
 'Ensembl_dbAccessionId_B',
 'Ensembl_dbExonId_A',
 'Ensembl_dbExonId_B',
 'Ensembl_dbTranscriptId_A',
 'Ensembl_dbTranscriptId_B',
 'Ensembl_dbTranslationId_A',
 'Ensembl_dbTranslationId_B',
 'Ensembl_regionEnd_A',
 'Ensembl_regionEnd_B',
 'Ensembl_regionId_A',
 'Ensembl_regionId_B',
 'Ensembl_regionResNum_A',
 'Ensembl_regionResNum_B',
 'Ensembl_regionStart_A',
 'Ensembl_regionStart_B',
 'INSCODE_A',
 'INSCODE_B',
 'Int_Types',
 'PDB_Annotation_A',
 'PDB_Annotation_B',
 'PDB_codeSecondaryStructure_A',
 'PDB_code

In [39]:
# Reads in pickle file from VarAlign output, containing all DSSP data
dssp = pd.read_pickle('C:\\Users\\Adam Martin\\OneDrive - University of Dundee\\Honours Project\\231006a-AM-honours-project-aps\\varalign-data-PF00026\\pickles\\mapped_dssp_table.p.gz')

In [40]:
# This combines the Structure and dssp table, on the protein accession abd protein chain.
merged = pd.merge(Structure, dssp, left_on=['PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A'],
                  right_on=['PDB_dbAccessionId', 'PDB_dbChainId', 'PDB_dbResNum'],
                  how='left')#, validate='m:1')


# Renames some columns for later analysis
merged.rename(columns={'SS': 'SS_A', 'SS_CLASS' : 'SS_CLASS_A', 'SS_CLASS_UNB': 'SS_CLASS_UNB_A',  'RSA': 'RSA_A', 'RSA_CLASS': 'RSA_CLASS_A', 'RSA_UNB' :'RSA_UNB_A', 'RSA_CLASS_UNB' : 'RSA_CLASS_UNB_A', }, inplace=True)

In [ ]:
# Fill in the gaps within the secondary structure assingment
merged.SS_A = merged.SS_A.fillna("") # FILL NA WITH "" SO WE CAN PROCESS THE SECONDARY STRUCTURE DATA. NA ARE FOUND ON THOSE RESIDUES PRESENTING NO SS, i.e., COIL
print(merged.SS_A)

In [ ]:
# Outputs the structure table as a feather file. This is a file format compatabile with python and R. 
merged.to_feather("C:\\Users\\Adam Martin\\OneDrive - University of Dundee\\Honours Project\\231006a-AM-honours-project-aps\\varalign-data-PF00026\\feather\\PF00026.22_swissprot.sto_prointvar_structure_table.p.gz.feather")

In [11]:
# This calculates the consequence sequences for the alignemnt. Selecting only columns with human sequence occupancy > 50% 
def get_cons_cols(aln_in, fmt_in, t = 0.50): #returns the columns with a relative occupancy greater than a threshold
    """
    """
    from Bio import AlignIO
    # Read the alignment
    alignment = AlignIO.read(aln_in, 'stockholm')
    human_indices = [i for i, seq_record in enumerate(alignment) if 'HUMAN' in seq_record.id]
    # Find positions with human residues in the alignment
    human_positions = [position + 1 for position in range(alignment.get_alignment_length()) if any(alignment[seq_index, position] != '-' for seq_index in human_indices)]
    # Print the positions with human residues
    print("Positions with human residues:", human_positions)
    print("Length:", len(human_positions))
    return human_positions
human_cons_cols = get_cons_cols(r'C:\Users\Adam Martin\OneDrive - University of Dundee\Honours Project\Jalview and Chimera using the alignment from PF00026 first from linux\Uniprot_Sequences_For_Alignment_To_Allow_VarAlign_TO\Uniprot_Annotated_Alignment127_PFAM.sto', 'stockholm') # GETS CONSENSUS COLUMNS FROM MSA (OCCUPANCY > 0.5)
print(human_cons_cols)

Positions with human residues: [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 158, 167, 168, 169, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 207, 208, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 255, 256, 257, 258, 259, 260, 261, 262, 263, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 386, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 425, 426, 427, 428

In [ ]:
# Code taken from https://github.com/bartongroup/JSU_ANK_analysis.
# Function that works out the concensus relative solvent accessability (RSA) for each of the 16 proteins, it is then combined together
def get_rsa_class_consensus(df, aln_cols):
    """
    """
    df_unique = df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A'])
    df_unique = df_unique.dropna(subset = ["RSA_CLASS_A", "RSA_A"])            
    df_unique = df_unique.query('RSA_A != ""')
    df_unique.Alignment_column_A = df_unique.Alignment_column_A.astype(float).astype(int)
    df_cons_cols = df_unique[df_unique.Alignment_column_A.isin(aln_cols)]

    grouped_col_rep = df_cons_cols.groupby(["Alignment_column_A", "SOURCE_ID_A"])
    rsa_class_dict = {}
    rsa_dict = {}
    i = 1
    for k, v in grouped_col_rep:
        col = k[0]
        if col not in rsa_class_dict:
            print("Processing consensus column {}".format(i))
            i += 1
            rsa_class_dict[col] = []
        if col not in rsa_dict:
            rsa_dict[col] = []
        
        rsa_list = list(map(float, v.RSA_A.tolist()))            
        rsa_class_list = v.RSA_CLASS_A.tolist()                 
        try:
            rsa_class_cons = statistics.mode(rsa_class_list)
        except:
            rsa_class_cons = rsa_class_list[0]
        rsa_median = statistics.median(rsa_list)
        rsa_class_dict[col].append(rsa_class_cons)
        rsa_dict[col].append(rsa_median)
    return rsa_class_dict, rsa_dict


rsa_class_dict, rsa_dict = get_rsa_class_consensus(merged, human_cons_cols) # RETRIEVES RSA DATA FROM TABLE
print(len(rsa_class_dict))
print(rsa_class_dict)



In [ ]:
# Code taken from https://github.com/bartongroup/JSU_ANK_analysis.
# Function that works out the relative solvent assessability assingment proportion for all the proteins
def get_rsa_class_df(rsa_class_dict, cons_cols_allsp):
    n = len(rsa_class_dict.keys())
    core = []
    surf = []
    part = []
    for v in rsa_class_dict.values():
        core.append(v.count("Core"))
        surf.append(v.count("Surface"))
        part.append(v.count("Part. Exposed")) 
    df_dssp = pd.DataFrame(list(zip(core, surf, part)), columns =  ["core", "surf", "part"])
    df_dssp["tot"] = df_dssp.core + df_dssp.surf + df_dssp.part
    df_dssp["p_core"] = df_dssp.core / df_dssp.tot
    df_dssp["p_surf"] = df_dssp.surf / df_dssp.tot
    df_dssp["p_part"] = df_dssp.part / df_dssp.tot
    df_dssp["p_tot"] = df_dssp.p_core + df_dssp.p_surf + df_dssp.p_part
    df_dssp.index = cons_cols_allsp         #
    return df_dssp


rsa_class_df = get_rsa_class_df(rsa_class_dict, human_cons_cols) # CALCULATES PROPORTION OF RSA CLASS PER POSITION
rsa_class_df

# Output the RSA class assingments proportions as a feather 
rsa_class_df.to_feather("/homes/2414054/Honours/231006a-AM-honours-project-aps/varalign-data-PF00026/feather/rsa_class_df.feather")

In [ ]:
# Code taken from https://github.com/bartongroup/JSU_ANK_analysis.
# Function that works out the concensus secondary structure for each of the 16 proteins, it is then combined together
def get_ss_class_consensus(df, aln_cols):
    """
    """
    df_unique = df.drop_duplicates(['UniProt_dbAccessionId_A', 'PDB_dbAccessionId_A', 'PDB_dbChainId_A', 'PDB_dbResNum_A'])
    df_unique = df_unique.dropna(subset = ["SS_CLASS_A"])
    df_unique.Alignment_column_A = df_unique.Alignment_column_A.astype(float).astype(int)
    df_cons_cols = df_unique[df_unique.Alignment_column_A.isin(aln_cols)]
    missing = []
    for x in aln_cols:
        if x in df_cons_cols['Alignment_column_A'].values:
            t = 0
        else: 
            missing.append(x)

    grouped_col_rep = df_cons_cols.groupby(["Alignment_column_A", "SOURCE_ID_A"])
    ss_class_dict = {}
    i = 1
    for k, v in grouped_col_rep:
        col = k[0]
        if col not in ss_class_dict:

            i += 1
            ss_class_dict[col] = []
        
        for x in missing:
            ss_class_dict[x] = []
            
        
        ss_class_list = v.SS_CLASS_A.tolist()

        try:
            ss_class_cons = statistics.mode(ss_class_list)
        except:
            ss_class_cons = ss_class_list[0]
        ss_class_dict[col].append(ss_class_cons)
        ss_class_dict = dict(sorted(ss_class_dict.items()))
    return ss_class_dict
ss_class_dict = get_ss_class_consensus(merged, human_cons_cols) # EXTRACTS SS DATA FROM TABLE AND RETURNS DICTIONARY WITH OCCURRENCE OF SS CLASS PER POSTITION
print(len(ss_class_dict))
print(ss_class_dict)

In [ ]:
# Code taken from https://github.com/bartongroup/JSU_ANK_analysis.
# Function that works out the secondary structure assingment proportion for all the proteins
def get_ss_class_df(rsa_class_dict):
    n = len(rsa_class_dict.keys())
    h = []
    c = []
    e = []
    for v in rsa_class_dict.values():
        h.append(v.count("H"))
        c.append(v.count("C"))
        e.append(v.count("E")) 
    df_dssp = pd.DataFrame(list(zip(h, c, e)), columns =  ["helix", "coil", "strand"])
    df_dssp["tot"] = df_dssp.helix + df_dssp.coil + df_dssp.strand
    df_dssp["p_helix"] = df_dssp.helix / df_dssp.tot
    df_dssp["p_coil"] = df_dssp.coil / df_dssp.tot
    df_dssp["p_strand"] = df_dssp.strand / df_dssp.tot
    df_dssp["p_tot"] = df_dssp.p_helix + df_dssp.p_coil + df_dssp.p_strand
    df_dssp.index = range(1, n + 1)
    return df_dssp
ss_class_df = get_ss_class_df(ss_class_dict) # CALCULATES PROPORTION OF EACH SECONDARY STRUCTURE CLASS PER POISITION
print(len(ss_class_df))
print(ss_class_df)

# Output the secondary structure class assingments proportions as a feather 
ss_class_df.to_feather("/homes/2414054/Honours/231006a-AM-honours-project-aps/varalign-data-PF00026/feather/ss_class_df.feather")


In [9]:
# Reads in pickle file from VarAlign output, containing all Varaint Information
variant_table = pd.read_pickle(r'C:\Users\Adam Martin\OneDrive - University of Dundee\Honours Project\231006a-AM-honours-project-aps\varalign-data-PF00026\pickles\PF00026.22_swissprot.sto_variants.p.gz')
aln_in = r"C:\\Users\\Adam Martin\\OneDrive - University of Dundee\\Honours Project\\231006a-AM-honours-project-aps\\varalign-data-PF00026\\alignemnt\\PF00026.22_swissprot.sto" # MSA
aln_fmt = "stockholm" # MSA FORMAT

In [12]:
# Formats the the table into only human consensus sequences with human occupancy on >50%
def format_variant_table(df, col_mask, vep_mask = ["missense_variant"], tab_format = "gnomad"):
    df_filt = df.copy(deep = True)
    df_filt.reset_index(inplace=True)
    if tab_format == "gnomad":
        df_filt.columns = [' '.join(col).strip() for col in df_filt.columns.tolist()]
    df_filt.columns = [col.lower().replace(" ", "_") for col in df_filt.columns.tolist()]
    df_filt = df_filt[df_filt.source_id.str.contains("HUMAN")]
    df_filt = df_filt.dropna(subset=["vep_consequence"])
    df_filt = df_filt[df_filt.vep_consequence.isin(vep_mask) ]
    df_filt = df_filt[df_filt.alignment_column.isin(col_mask)]
    return df_filt
human_missense_variants = format_variant_table(variant_table, human_cons_cols) # GET ONLY MISSENSE VARIANTS ROWS
human_missense_variants

,source_id,protein_position,alignment_column,site,allele_num,feature,row_id,row_chrom,row_pos,row_ref,...,vep_pubmed,vep_motif_name,vep_motif_pos,vep_high_inf_pos,vep_motif_score_change,vep_lof,vep_lof_filter,vep_lof_flags,vep_lof_info,external_source_accession
6618,NAPSA_HUMAN/77-401,133,154,423.0,0.0,ENST00000253719,None,19,50864990.0,T,...,,,,,,,,,,O96009
6626,NAPSA_HUMAN/77-401,141,168,417.0,0.0,ENST00000253719,None,19,50864967.0,A,...,,,,,,,,,,O96009
6630,NAPSA_HUMAN/77-401,145,173,413.0,0.0,ENST00000253719,rs746693112,19,50864954.0,C,...,,,,,,,,,,O96009
6631,NAPSA_HUMAN/77-401,145,173,414.0,0.0,ENST00000253719,rs186910155,19,50864955.0,G,...,,,,,,,,,,O96009
6639,NAPSA_HUMAN/77-401,151,180,410.0,1.0,ENST00000253719,rs773392864,19,50864935.0,G,...,,,,,,,,,,O96009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41519,PEPC_HUMAN/72-387,323,646,738.0,0.0,ENST00000373025,rs369120707,6,41705513.0,G,...,,,,,,,,,,P20142
41548,PEPC_HUMAN/72-387,346,719,692.0,0.0,ENST00000373025,None,6,41704720.0,C,...,,,,,,,,,,P20142
41551,PEPC_HUMAN/72-387,347,720,691.0,0.0,ENST00000373025,rs187120335,6,41704718.0,C,...,,,,,,,,,,P20142
41557,PEPC_HUMAN/72-387,353,728,687.0,0.0,ENST00000373025,rs768749667,6,41704700.0,A,...,,,,,,,,,,P20142


In [ ]:
# Output the human missense variants table as a feather 
human_missense_variants.to_feather("/homes/2414054/Honours/231006a-AM-honours-project-aps/varalign-data-PF00026/feather/PF00026.22_swissprot.sto_variants.p.gz.feather")
